<a href="https://colab.research.google.com/github/renan-peres/mfin-portfolio-management/blob/main/02_bond_portfolio_contruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bond Portfolio Selection
Bond selection with convexity > 1 and higher price sensitivity to changes in interest rates.

### Step 1: Import Libraries

In [1]:
# UDFs
from py.utils import load_and_filter_data, export_to_excel
from py.bond_selection import calculate_bond_price, add_bond_prices_to_df, calculate_duration_for_bonds, calculate_modified_duration, calculate_price_change_sensitivity

# Data manipulation libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Excel libraries
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import os

### Step 2: Define Parameters

#### Dates

In [2]:
# Define the date range
end_date = (datetime.today() - BDay(1)).to_pydatetime()  # Subtract 1 business day
# end_date = pd.to_datetime('2025-04-26')                # Report date
start_date = end_date - timedelta(days=5*365)

# Convert datetime objects to Unix timestamps (seconds since Jan 1, 1970)
start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

# Print the date range
days_difference = (end_date - start_date).days
print(f"Date Range: {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
print(f"Time span: {days_difference} days ({days_difference/365:.2f} years)")

Date Range: 2020-05-31 to 2025-05-30
Time span: 1825 days (5.00 years)


#### Risk-free rate (T-bill, %)

In [3]:
# Load and process data
daily_risk_free_df = load_and_filter_data('data/datasets/daily_treasury_rates.csv', ['^IRX'], start_date, end_date)
# risk_free_rate = .0433 
risk_free_rate = daily_risk_free_df.iloc[-1, 0] / 100

# Display result
print("Risk-Free Rate:", risk_free_rate, "-- 13 WEEK TREASURY BILL (^IRX)")
daily_risk_free_df.head()

Found 1 of 1 tickers in data/datasets/daily_treasury_rates.csv
Missing tickers: []
Risk-Free Rate: 0.042300000000000004 -- 13 WEEK TREASURY BILL (^IRX)


,^IRX
Date,
2020-06-01,0.128
2020-06-02,0.145
2020-06-03,0.148
2020-06-04,0.140
2020-06-05,0.145


#### Report File

In [4]:
output_file = f'reports/portfolio-{datetime.date(end_date)}.xlsx'

### Step 3: Import Bond Data

In [5]:
blackrock_etf_df = pd.read_csv('data/datasets/fundamentals_blackrock_bonds.csv')
blackrock_etf_df.head()

,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,AGG,iShares Core U.S. Aggregate Bond ETF,3.16,8.00,1.96,-0.68,1.50,3.10,"Apr 30, 2025","Sep 22, 2003",123886933466,239458,https://www.blackrock.com/us/individual/produc...,0.0481,8.10,0.52,0.0355
1,AGIH,iShares Inflation Hedged U.S. Aggregate Bond ETF,3.42,7.17,-,-,-,2.72,"Apr 30, 2025","Jun 22, 2022",2462074,328179,https://www.blackrock.com/us/individual/produc...,0.0476,7.79,0.51,NaN
2,AGRH,iShares Interest Rate Hedged U.S. Aggregate Bo...,0.77,4.50,-,-,-,5.19,"Apr 30, 2025","Jun 22, 2022",7752588,328180,https://www.blackrock.com/us/individual/produc...,0.0532,8.11,-0.16,NaN
3,AGZ,iShares Agency Bond ETF,2.85,6.98,2.84,0.48,1.74,2.34,"Apr 30, 2025","Nov 05, 2008",610222073,239457,https://www.blackrock.com/us/individual/produc...,0.0431,4.11,0.24,0.0347
4,BAIPX,iShares Short-Term TIPS Bond Index Fund,3.76,7.54,3.19,3.67,-,2.83,"Apr 30, 2025","Feb 16, 2016",5159871,282302,https://www.blackrock.com/us/individual/produc...,NaN,NaN,NaN,NaN


### Step 4: Filter for Convexity > 1

In [6]:
bond_funds_filtered_df = blackrock_etf_df[blackrock_etf_df['Convexity'] >= 1].sort_values(by='Convexity', ascending=False).reset_index(drop=True)

# Define all possible percentage columns
all_percentage_cols = ['YTD (%)', '1Y (%)', '3Y (%)', '5Y (%)', '10Y (%)', 'Incept (%)', 'Since Inception (%)']

# Filter to only include columns that actually exist in the DataFrame
percentage_cols = [col for col in all_percentage_cols if col in bond_funds_filtered_df.columns]

print(f"Processing the following percentage columns: {percentage_cols}")

# First, ensure all percentage columns are converted to numeric values
for col in percentage_cols:
    # Convert to numeric first, handling errors by setting them to NaN
    bond_funds_filtered_df[col] = pd.to_numeric(bond_funds_filtered_df[col], errors='coerce')

# Now perform the division safely
for col in percentage_cols:
    bond_funds_filtered_df[col] = bond_funds_filtered_df[col] / 100

# Format as percentage strings
for col in percentage_cols + ['Yield_To_Maturity', 'Weighted_Avg_Coupon']:
    # Check if column exists and only process if it does
    if col in bond_funds_filtered_df.columns:
        # Only format cells that aren't NaN
        bond_funds_filtered_df[col] = bond_funds_filtered_df[col].apply(
            lambda x: '{:.2%}'.format(x) if pd.notna(x) else x
        )

bond_tickers = bond_funds_filtered_df['Ticker'].tolist()

# Display Outputs
print(f"Number of iShares bond/fixed income funds with Convexity >= 1: {len(bond_tickers)}")
print(bond_tickers)
display(bond_funds_filtered_df.head())

Processing the following percentage columns: ['YTD (%)', '1Y (%)', '3Y (%)', '5Y (%)', '10Y (%)', 'Incept (%)']
Number of iShares bond/fixed income funds with Convexity >= 1: 13
['GOVZ', 'IBGL', 'TLT', 'IBGB', 'ILTB', 'IGLB', 'IBGA', 'TLH', 'ICVT', 'IGOV', 'LQD', 'ELQD', 'LQDI']


,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,GOVZ,iShares 25+ Year Treasury STRIPS Bond ETF,0.65%,1.05%,-12.71%,NaN,NaN,-15.79%,"Apr 30, 2025","Sep 22, 2020",261525314,315911,https://www.blackrock.com/us/individual/produc...,5.09%,27.26,7.22,0.00%
1,IBGL,iShares® iBonds® Dec 2055 Term Treasury ETF,NaN,NaN,NaN,NaN,NaN,NaN,"Apr 30, 2025","Mar 25, 2025",3606540,342146,https://www.blackrock.com/us/individual/produc...,4.99%,29.71,3.43,4.63%
2,TLT,iShares 20+ Year Treasury Bond ETF,3.23%,5.46%,-6.03%,-9.43%,-0.87%,3.79%,"Apr 30, 2025","Jul 22, 2002",50539489865,239454,https://www.blackrock.com/us/individual/produc...,5.05%,25.47,3.35,2.87%
3,IBGB,iShares® iBonds® Dec 2045 Term Treasury ETF,NaN,NaN,NaN,NaN,NaN,NaN,"Apr 30, 2025","Mar 25, 2025",3626020,342124,https://www.blackrock.com/us/individual/produc...,5.06%,19.76,2.32,3.18%
4,ILTB,iShares Core 10+ Year USD Bond ETF,2.20%,6.73%,-1.03%,-3.92%,1.47%,3.87%,"Apr 30, 2025","Dec 08, 2009",592240607,239424,https://www.blackrock.com/us/individual/produc...,5.72%,21.71,2.32,3.94%


In [7]:
# Ensure relevant columns are numeric
bond_funds_filtered_df['Yield_To_Maturity'] = pd.to_numeric(
    bond_funds_filtered_df['Yield_To_Maturity'].str.replace('%', ''), errors='coerce'
)
bond_funds_filtered_df['Weighted_Avg_Coupon'] = pd.to_numeric(
    bond_funds_filtered_df['Weighted_Avg_Coupon'].str.replace('%', ''), errors='coerce'
)
bond_funds_filtered_df['Weighted_Avg_Maturity'] = pd.to_numeric(
    bond_funds_filtered_df['Weighted_Avg_Maturity'], errors='coerce'
)

# Fill missing values with 0
bond_funds_filtered_df = bond_funds_filtered_df.fillna(0)

# Divide the columns by 100 and reassign
bond_funds_filtered_df['Yield_To_Maturity'] = bond_funds_filtered_df['Yield_To_Maturity'] / 100
bond_funds_filtered_df['Weighted_Avg_Coupon'] = bond_funds_filtered_df['Weighted_Avg_Coupon'] / 100

# Display the DataFrame with the updated columns
display(bond_funds_filtered_df.head())

,Ticker,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,Product_ID,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon
0,GOVZ,iShares 25+ Year Treasury STRIPS Bond ETF,0.65%,1.05%,-12.71%,0,0,-15.79%,"Apr 30, 2025","Sep 22, 2020",261525314,315911,https://www.blackrock.com/us/individual/produc...,0.0509,27.26,7.22,0.0000
1,IBGL,iShares® iBonds® Dec 2055 Term Treasury ETF,0,0,0,0,0,0,"Apr 30, 2025","Mar 25, 2025",3606540,342146,https://www.blackrock.com/us/individual/produc...,0.0499,29.71,3.43,0.0463
2,TLT,iShares 20+ Year Treasury Bond ETF,3.23%,5.46%,-6.03%,-9.43%,-0.87%,3.79%,"Apr 30, 2025","Jul 22, 2002",50539489865,239454,https://www.blackrock.com/us/individual/produc...,0.0505,25.47,3.35,0.0287
3,IBGB,iShares® iBonds® Dec 2045 Term Treasury ETF,0,0,0,0,0,0,"Apr 30, 2025","Mar 25, 2025",3626020,342124,https://www.blackrock.com/us/individual/produc...,0.0506,19.76,2.32,0.0318
4,ILTB,iShares Core 10+ Year USD Bond ETF,2.20%,6.73%,-1.03%,-3.92%,1.47%,3.87%,"Apr 30, 2025","Dec 08, 2009",592240607,239424,https://www.blackrock.com/us/individual/produc...,0.0572,21.71,2.32,0.0394


### Step 5: Import Quotes

In [8]:
bond_tickers = bond_funds_filtered_df['Ticker'].tolist()
print(bond_tickers)

['GOVZ', 'IBGL', 'TLT', 'IBGB', 'ILTB', 'IGLB', 'IBGA', 'TLH', 'ICVT', 'IGOV', 'LQD', 'ELQD', 'LQDI']


In [9]:
# Load and process data
bonds_daily_df = load_and_filter_data('data/datasets/daily_bond_quotes.csv', bond_tickers, start_date, end_date)
display(bonds_daily_df.head())

Found 10 of 13 tickers in data/datasets/daily_bond_quotes.csv
Missing tickers: ['IBGL', 'IBGB', 'IBGA']


,ELQD,GOVZ,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,,,
2020-06-01,NaN,NaN,55.83,55.29,49.80,60.42,111.57,21.71,143.42,141.83
2020-06-02,NaN,NaN,56.43,55.64,49.89,60.42,111.92,22.04,143.07,141.31
2020-06-03,NaN,NaN,57.17,55.33,49.98,60.00,111.86,21.83,141.49,139.43
2020-06-04,NaN,NaN,56.58,55.15,50.18,59.77,111.52,21.91,140.14,137.41
2020-06-05,NaN,NaN,57.57,55.70,50.01,59.82,112.09,21.87,139.22,136.44


### Step 6: Prepare Data (Drop Invalid Tickers)

#### Daily Quotes

In [10]:
# Identify columns with null values in first or last 50 rows
first_50_nulls = bonds_daily_df.head(50).isnull().any()
last_50_nulls = bonds_daily_df.tail(50).isnull().any()

# Columns to drop are those with nulls in first 50 OR last 50 rows
columns_to_drop = first_50_nulls | last_50_nulls
bad_columns = columns_to_drop[columns_to_drop].index.tolist()

print(f"Dropping {len(bad_columns)} columns with missing values in first/last 50 records: {bad_columns}")

# Drop those columns
bonds_daily_filtered_df = bonds_daily_df.loc[:, ~columns_to_drop]

# Display the cleaned dataframe
display(bonds_daily_filtered_df.head())

Dropping 2 columns with missing values in first/last 50 records: ['ELQD', 'GOVZ']


,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-06-01,55.83,55.29,49.80,60.42,111.57,21.71,143.42,141.83
2020-06-02,56.43,55.64,49.89,60.42,111.92,22.04,143.07,141.31
2020-06-03,57.17,55.33,49.98,60.00,111.86,21.83,141.49,139.43
2020-06-04,56.58,55.15,50.18,59.77,111.52,21.91,140.14,137.41
2020-06-05,57.57,55.70,50.01,59.82,112.09,21.87,139.22,136.44


#### Monthly Quotes

In [11]:
bonds_monthly_filtered_df = (bonds_daily_filtered_df.set_index(pd.to_datetime(bonds_daily_filtered_df.pop('Date')))
      if 'Date' in bonds_daily_filtered_df.columns else bonds_daily_filtered_df.copy())
bonds_monthly_filtered_df.index = pd.to_datetime(bonds_monthly_filtered_df.index)              
bonds_monthly_filtered_df = (bonds_monthly_filtered_df.resample('MS').last()
   .reset_index()
   .rename(columns={'index': 'Date'}))

bonds_monthly_filtered_df.set_index('Date', inplace=True)
display(bonds_monthly_filtered_df.head())

,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-06-01,59.46,57.22,50.36,61.71,113.99,22.55,144.24,143.27
2020-07-01,63.86,60.35,52.75,65.17,117.53,23.49,148.27,149.62
2020-08-01,70.40,58.16,52.80,62.75,115.44,23.77,142.85,142.07
2020-09-01,67.84,58.00,52.31,62.61,114.93,23.62,143.94,143.17
2020-10-01,68.06,57.35,52.41,61.50,114.34,23.75,139.90,138.32


### Step 7: Calculate Returns (Lognormal)

In [12]:
log_returns_df = np.log(bonds_daily_filtered_df / bonds_daily_filtered_df.shift(1))
log_returns_df = log_returns_df.dropna().sort_index(axis=0, ascending=True).sort_index(axis=1, ascending=True)
log_returns_df.head()

,ICVT,IGLB,IGOV,ILTB,LQD,LQDI,TLH,TLT
Date,,,,,,,,
2020-06-02,0.010690,0.006310,0.001806,0.000000,0.003132,0.015086,-0.002443,-0.003673
2020-06-03,0.013028,-0.005587,0.001802,-0.006976,-0.000536,-0.009574,-0.011105,-0.013393
2020-06-04,-0.010374,-0.003259,0.003994,-0.003841,-0.003044,0.003658,-0.009587,-0.014594
2020-06-05,0.017346,0.009923,-0.003394,0.000836,0.005098,-0.001827,-0.006587,-0.007084
2020-06-08,0.015683,0.003942,0.005782,0.006000,0.003384,0.014075,0.001220,0.003804


### Step 8: Calculate Bond Metrics

#### Bond Price (PV)

In [13]:
# Apply the function to your DataFrame
bond_funds_price_df = add_bond_prices_to_df(bond_funds_filtered_df)

# Display the updated DataFrame with bond prices
bond_funds_price_df[['Ticker', 'Weighted_Avg_Maturity', 'Weighted_Avg_Coupon', 
                      'Yield_To_Maturity', 'Bond_Price']]

,Ticker,Weighted_Avg_Maturity,Weighted_Avg_Coupon,Yield_To_Maturity,Bond_Price
0,GOVZ,27.26,0.000,0.051,25.406
1,IBGL,29.71,0.046,0.050,94.454
2,TLT,25.47,0.029,0.050,68.952
3,IBGB,19.76,0.032,0.051,76.687
4,ILTB,21.71,0.039,0.057,78.028
5,IGLB,22.09,0.046,0.060,82.654
6,IBGA,18.83,0.040,0.050,87.428
7,TLH,16.93,0.032,0.050,79.429
8,ICVT,2.92,0.021,0.031,97.367
9,IGOV,9.55,0.022,0.029,94.685


#### Modified Duration (D*)

In [14]:
bond_tickers = bonds_daily_filtered_df.columns.tolist()
bond_funds_filtered_df = bond_funds_filtered_df[bond_funds_filtered_df['Ticker'].isin(bond_tickers)].reset_index(drop=True)

# Apply the function to your DataFrame
bond_funds_duration_df = calculate_duration_for_bonds(bond_funds_filtered_df, end_date)

# Display the updated DataFrame
print(bond_tickers)
bond_funds_duration_df[['Ticker', 'Weighted_Avg_Maturity', 'Weighted_Avg_Coupon', 'Yield_To_Maturity', 'Bond_Price', 'Duration (D*)', 'Convexity']]

['ICVT', 'IGLB', 'IGOV', 'ILTB', 'LQD', 'LQDI', 'TLH', 'TLT']


,Ticker,Weighted_Avg_Maturity,Weighted_Avg_Coupon,Yield_To_Maturity,Bond_Price,Duration (D*),Convexity
0,TLT,25.47,0.029,0.050,68.952,17.815,3.35
1,ILTB,21.71,0.039,0.057,78.028,14.865,2.32
2,IGLB,22.09,0.046,0.060,82.654,14.570,2.16
3,TLH,16.93,0.032,0.050,79.429,12.854,1.86
4,ICVT,2.92,0.021,0.031,97.367,2.411,1.17
5,IGOV,9.55,0.022,0.029,94.685,8.532,1.11
6,LQD,12.80,0.044,0.054,90.586,9.738,1.10
7,LQDI,12.43,0.000,0.053,52.320,11.691,1.06


#### Price Sensitivity to Changes in YTM (-1%)

In [15]:
# Apply the function to your DataFrame
bond_funds_sensitivity_df = calculate_price_change_sensitivity(bond_funds_duration_df)
bond_funds_sensitivity_df[['Ticker', 'Bond_Price', 'Duration (D*)', 'Convexity', 'Price Sensitivity to YTM (-1%)']]

,Ticker,Bond_Price,Duration (D*),Convexity,Price Sensitivity to YTM (-1%)
0,TLT,68.952,17.815,3.35,0.1782
1,ILTB,78.028,14.865,2.32,0.1487
2,IGLB,82.654,14.570,2.16,0.1457
3,TLH,79.429,12.854,1.86,0.1285
4,ICVT,97.367,2.411,1.17,0.0241
5,IGOV,94.685,8.532,1.11,0.0853
6,LQD,90.586,9.738,1.10,0.0974
7,LQDI,52.320,11.691,1.06,0.1169


### Step 9: Select the Best Bond (Highest Price Sentivity to Changes in YTM)

In [16]:
# Find the row index where Price Sensitivity to YTM (-1%) is at its maximum
best_bond_ticker = bond_funds_sensitivity_df["Price Sensitivity to YTM (-1%)"].idxmax()

# Get the maximum value for display
max_sensitivity = bond_funds_sensitivity_df["Price Sensitivity to YTM (-1%)"].max()

print(f"Best Bond Ticker: {best_bond_ticker}")
print(f"Maximum Price Sensitivity to YTM (-1%): {max_sensitivity:.6f}")
bond_funds_sensitivity_df.loc[best_bond_ticker]

Best Bond Ticker: 0
Maximum Price Sensitivity to YTM (-1%): 0.178200


Ticker                                                                          TLT
Name                                             iShares 20+ Year Treasury Bond ETF
YTD (%)                                                                       3.23%
1Y (%)                                                                        5.46%
3Y (%)                                                                       -6.03%
5Y (%)                                                                       -9.43%
10Y (%)                                                                      -0.87%
Incept (%)                                                                    3.79%
Perf. as of                                                            Apr 30, 2025
Inception Date                                                         Jul 22, 2002
Net Assets                                                              50539489865
Product_ID                                                                  

### Step 10: Add Returns and Standard Deviation to DataFrame

In [17]:
# Step 1: Unpivot (melt) the log_returns_df
log_returns_long = log_returns_df.reset_index().melt(id_vars='Date', var_name='Ticker', value_name='Log Return')

# Step 2: Ensure 'Log Return' column is numeric
log_returns_long['Log Return'] = pd.to_numeric(log_returns_long['Log Return'], errors='coerce')

# Step 3: Drop rows with NaN values in 'Log Return'
log_returns_long = log_returns_long.dropna(subset=['Log Return'])

# Step 4: Calculate statistics for each ticker
log_returns_stats = log_returns_long.groupby('Ticker').agg(
    Expected_Return=('Log Return', lambda x: ((1 + x).prod() ** (252 / len(x))) - 1),  # Geometric mean (annualized return)
    Standard_Deviation=('Log Return', lambda x: x.std() * np.sqrt(252))  # Annualized standard deviation
).reset_index()

# Step 6: Set Index
log_returns_stats.set_index('Ticker', inplace=True)

# Step 7: Merge with bond_funds_sensitivity_df
bond_funds_sensitivity_df = bond_funds_sensitivity_df.merge(
    log_returns_stats,
    on='Ticker',
    how='left'
)

# Step 7: Set Index
bond_funds_sensitivity_df.set_index('Ticker', inplace=True)

# Display the updated DataFrame
display(bond_funds_sensitivity_df)

,Name,YTD (%),1Y (%),3Y (%),5Y (%),10Y (%),Incept (%),Perf. as of,Inception Date,Net Assets,...,URL,Yield_To_Maturity,Weighted_Avg_Maturity,Convexity,Weighted_Avg_Coupon,Bond_Price,Duration (D*),Price Sensitivity to YTM (-1%),Expected_Return,Standard_Deviation
Ticker,,,,,,,,,,,,,,,,,,,,,
TLT,iShares 20+ Year Treasury Bond ETF,3.23%,5.46%,-6.03%,-9.43%,-0.87%,3.79%,"Apr 30, 2025","Jul 22, 2002",50539489865,...,https://www.blackrock.com/us/individual/produc...,0.050,25.47,3.35,0.029,68.952,17.815,0.1782,-0.107146,0.162526
ILTB,iShares Core 10+ Year USD Bond ETF,2.20%,6.73%,-1.03%,-3.92%,1.47%,3.87%,"Apr 30, 2025","Dec 08, 2009",592240607,...,https://www.blackrock.com/us/individual/produc...,0.057,21.71,2.32,0.039,78.028,14.865,0.1487,-0.052162,0.128356
IGLB,iShares 10+ Year Investment Grade Corporate Bo...,1.10%,6.19%,0.87%,-1.95%,2.09%,4.27%,"Apr 30, 2025","Dec 08, 2009",2403178454,...,https://www.blackrock.com/us/individual/produc...,0.060,22.09,2.16,0.046,82.654,14.570,0.1457,-0.032403,0.127069
TLH,iShares 10-20 Year Treasury Bond ETF,4.02%,7.73%,-2.63%,-6.71%,-0.35%,2.95%,"Apr 30, 2025","Jan 05, 2007",10974997953,...,https://www.blackrock.com/us/individual/produc...,0.050,16.93,1.86,0.032,79.429,12.854,0.1285,-0.078712,0.128818
ICVT,iShares Convertible Bond ETF,0.45%,13.09%,5.30%,10.75%,0,9.08%,"Apr 30, 2025","Jun 02, 2015",2316039188,...,https://www.blackrock.com/us/individual/produc...,0.031,2.92,1.17,0.021,97.367,2.411,0.0241,0.080581,0.150239
IGOV,iShares International Treasury Bond ETF,8.65%,9.56%,-0.71%,-3.32%,-0.86%,0.21%,"Apr 30, 2025","Jan 21, 2009",1016521594,...,https://www.blackrock.com/us/individual/produc...,0.029,9.55,1.11,0.022,94.685,8.532,0.0853,-0.039034,0.096500
LQD,iShares iBoxx $ Investment Grade Corporate Bon...,2.22%,7.50%,2.61%,-0.17%,2.37%,4.41%,"Apr 30, 2025","Jul 22, 2002",30741667860,...,https://www.blackrock.com/us/individual/produc...,0.054,12.80,1.10,0.044,90.586,9.738,0.0974,-0.011331,0.087882
LQDI,iShares Inflation Hedged Corporate Bond ETF,2.32%,6.38%,2.12%,4.01%,0,3.95%,"Apr 30, 2025","May 08, 2018",90498810,...,https://www.blackrock.com/us/individual/produc...,0.053,12.43,1.06,0.000,52.320,11.691,0.1169,0.031790,0.087521


### Step 11: Export Data to Excel

In [18]:
# Check if best_bond_ticker is a numeric index instead of ticker name
if isinstance(best_bond_ticker, (int, np.integer)):
    # Get the actual ticker name from the dataframe
    if 'Ticker' in bond_funds_sensitivity_df.columns:
        best_bond_ticker = bond_funds_sensitivity_df.iloc[best_bond_ticker]['Ticker']
        print(f"Using ticker name: {best_bond_ticker}")
    else:
        # If Ticker is the index name
        best_bond_ticker = bond_funds_sensitivity_df.index[best_bond_ticker]
        print(f"Using ticker name from index: {best_bond_ticker}")

export_to_excel(output_file, {
    'bond': bond_funds_sensitivity_df,
    'daily_quotes': bonds_daily_filtered_df[[best_bond_ticker]],  
    'monthly_quotes': bonds_monthly_filtered_df[[best_bond_ticker]]   
})

Using ticker name from index: TLT
Updated sheet 'bond'
Successfully merged data into 'daily_quotes' sheet
Successfully merged data into 'monthly_quotes' sheet
Successfully exported all data to reports/portfolio-2025-05-30.xlsx
